In [ ]:
import torch
import torch.nn as nn

class LinearSVM(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.linear = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.linear(x)
def multiclass_hinge_loss(outputs, labels, margin=1.0):
    """
    outputs: (batch_size, num_classes)
    labels: (batch_size,) — correct class index
    """
    batch_size = outputs.size(0)
    correct_class_scores = outputs[torch.arange(batch_size), labels].unsqueeze(1)
    
    margins = outputs - correct_class_scores + margin
    margins[torch.arange(batch_size), labels] = 0  # ignore correct class
    loss = torch.clamp(margins, min=0).sum() / batch_size
    return loss


In [ ]:
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

X = np.load("features/six_classes/fc7.npy")      
y = np.load("features/six_classes/labels.npy")

# Wrap in PyTorch DataLoader
X_tensor = torch.from_numpy(X).float()
y_tensor = torch.from_numpy(y).long()

dataset = TensorDataset(X_tensor, y_tensor)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

# Instantiate model
input_dim = X.shape[1]
num_classes = len(np.unique(y))
model = LinearSVM(input_dim, num_classes)

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Training loop
for epoch in range(10):
    total_loss = 0
    for xb, yb in loader:
        logits = model(xb)
        loss = multiclass_hinge_loss(logits, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Epoch 1, Loss: 35.2941
Epoch 2, Loss: 0.2121
Epoch 3, Loss: 0.0906
Epoch 4, Loss: 0.0000
Epoch 5, Loss: 0.0000
Epoch 6, Loss: 0.0000
Epoch 7, Loss: 0.0000
Epoch 8, Loss: 0.0000
Epoch 9, Loss: 0.0000
Epoch 10, Loss: 0.0000


In [4]:
with torch.no_grad():
    logits = model(X_tensor)
    predictions = torch.argmax(logits, dim=1)
    acc = (predictions == y_tensor).float().mean()
    print(f"Train Accuracy: {acc.item()*100:.2f}%")

Train Accuracy: 100.00%


In [5]:
dummy_input = torch.randn(1, input_dim)
torch.onnx.export(model, dummy_input, "svm_fc7_dynamic.onnx",
                  input_names=['input'],
                  output_names=['logits'],
                  dynamic_axes={'input': {0: 'batch_size'}, 'logits': {0: 'batch_size'}},
                  opset_version=11)